In [298]:
# all libraries
from PIL import Image
import numpy as np
from scipy.fftpack import dct
import heapq
from collections import defaultdict
import os

- Image Processing

In [299]:
def image_process(image_path):

    image = Image.open(image_path)

    image_array = np.array(image)

    # result_array = image_path - 128
    result_array = image_array - 128
    print(result_array)
    pad_height = (8 - (result_array.shape[0] % 8)) % 8
    pad_width = (8 - (result_array.shape[1] % 8)) % 8


    padded_image = np.pad(result_array, ((0, pad_height), (0, pad_width)), mode="constant", constant_values=0)

    return padded_image

- DCT computation

In [300]:
def compute_DCT(image, qmat):
    dct_coeff_quant = np.zeros((8, image.shape[1]*image.shape[0]//8))
    for i in range(image.shape[0]//8):
        for j in range(image.shape[1]//8):
            block = image[8*i:8*i+8, 8*j: 8*j+8]
            dct_coeff_quant[:, image.shape[1]*i + 8*j : image.shape[1]*i + 8*(j+1)] = np.round(dct(dct(block.T, norm="ortho").T, norm="ortho") / qmat).astype(int)
    print(np.max(dct_coeff_quant))

    return dct_coeff_quant.astype(int)
    

- Huffman encoding 

In [301]:
class HuffmanNode:
    def __init__(self, symbol, freq):
        self.symbol = symbol
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(freq_table):
    heap = [HuffmanNode(sym, freq) for sym, freq in freq_table.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = HuffmanNode(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)
    return heap[0]

def generate_huffman_codes(freq_table):
    codes = {}
    tree = build_huffman_tree(freq_table)
    def generate_codes_helper(node, current_code):
        if node.symbol is not None:
            codes[node.symbol] = current_code
            return
        generate_codes_helper(node.left, current_code + "0")
        generate_codes_helper(node.right, current_code + "1")

    generate_codes_helper(tree, "")
    return codes

- Run length Encoding

In [302]:
def zigzag_order(n):
    indices = [(x, y) for x in range(n) for y in range(n)]
    zigzag_indices = sorted(indices, key=lambda x: (x[0] + x[1], -x[0] if (x[0] + x[1]) % 2 == 0 else x[0]))
    # return [block[x, y] for x, y in zigzag_indices]
    return zigzag_indices

def run_length_encode(zigzag_block, huffman_codes):
    encoded = []
    run_length = 0

    for coefficient in zigzag_block:
        if coefficient == 0:
            run_length += 1
        else:
            size = len(huffman_codes[coefficient])  # Number of bits in Huffman code
            while run_length > 15:  # Handle long runs of zeros
                encoded.append((15, 0, None))  # Special code for run of 16 zeros
                run_length -= 15
            encoded.append((run_length, size, huffman_codes[coefficient]))
            run_length = 0

    # Add end-of-block (EOB) symbol if needed
    if run_length > 0:
        encoded.append((0, 0, None))  # EOB marker
    return encoded

- Entropy encoding

In [303]:
def encode(image_path, qmat):

    image = image_process(image_path)

    dct = compute_DCT(image, qmat)

    # calc diff
    dct_diff = dct - np.roll(dct, 8, axis=1)
    dct_diff[:,:8] = dct[:,:8]
    
    # get huffman for all blocks
    print(type(dct[0,0]))
    print(type(dct_diff[0,0]))
    unique_values, counts = np.unique(dct_diff, return_counts=True)
    freq_table = dict(zip(unique_values, counts))
    freq_table.pop(0, None)
    huffman_codes = generate_huffman_codes(freq_table)
    encoded = []
    # run length code
    zigzag_ind = zigzag_order(8)
    for i in range(dct_diff.shape[1]//8):
        block = dct_diff[:, 8*i:8*i+8]
        zigzag_block = [block[x, y] for x, y in zigzag_ind]
        encoded_block = run_length_encode(zigzag_block, huffman_codes)
        encoded += encoded_block
    
    return encoded, huffman_codes


- Saving in a file

In [304]:
def write_compressed_file(filename, quant_matrix, huffman_table, encoded_data):
    with open(filename, 'wb') as f:
        np.save(f, quant_matrix)
        np.save(f, huffman_table)
        np.save(f, encoded_data)
        
    file_size = os.path.getsize(filename)
    print(f"Size of the file '{filename}': {file_size} bytes")

In [ ]:
import numpy as np
import struct
from bitstring import BitArray
def write_compressed_file(filename, quant_matrix, huffman_table, encoded_data, is_color=True):
    with open(filename, 'wb') as f:
        
        file_size = 0  
        f.write(struct.pack('I', file_size))
        
        
        color_flag = 1 if is_color else 0
        f.write(struct.pack('B', color_flag))
        
        huffman_list = list(huffman_table.items())
        huff_bstream = BitArray()
        f.write(struct.pack('I', len(huffman_list)))  # Write the number of entries
        
        for coeff, huff_code in huffman_list:
            coeff_bit = BitArray(int=coeff, length=12)
            huff_len_bit = BitArray(uint=len(huff_code), length=4)
            huff_bit = BitArray(bin=huff_code)
            huff_bstream += coeff_bit + huff_len_bit + huff_bit

        huff_bstream.tofile(f)
        
        
        # Write the quantization matrix (assuming it's a 2D NumPy array)
        quant_matrix_size = quant_matrix.shape[0] * quant_matrix.shape[1]
        f.write(struct.pack('I', quant_matrix_size))  # Total elements in the matrix
        quant_matrix.tofile(f)  # Write the quantization matrix as raw binary data
        
        # Write the encoded data (bit stream)
        f.write(struct.pack('I', len(encoded_data)))  # Length of the encoded stream
        final_bitstream = BitArray()
        bitstream_len = 0
        for run_len, code_len, huff_code in encoded_data:
            run_len_bit = BitArray(uint=run_len, length=4)
            code_len_bit = BitArray(uint=code_len, length=4)
            if huff_code == None:
                bitstream_len += 8
                continue
            huff_code_bit = BitArray(bin=huff_code)
            final_bitstream += run_len_bit + code_len_bit + huff_code_bit
            bitstream_len += 8 + code_len
        final_bitstream.tofile(f)
        # Update the file size at the beginning
        current_position = f.tell()  # Current position after writing
        f.seek(0)  # Go back to the start of the file
        f.write(struct.pack('I', current_position))  # Write the actual file size
        f.seek(current_position)  # Return to the end of the file to continue writing

# Example usage
quant_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61], 
                         [12, 12, 14, 19, 26, 58, 60, 55], 
                         [14, 13, 16, 24, 40, 57, 69, 56], 
                         [14, 17, 22, 29, 51, 87, 80, 62], 
                         [18, 22, 37, 56, 68, 109, 103, 77], 
                         [24, 35, 55, 64, 81, 104, 113, 92], 
                         [49, 64, 78, 87, 103, 121, 120, 101], 
                         [72, 92, 95, 98, 112, 100, 103, 99]])

# Huffman table (example: DCT coefficient and Huffman code)
# huffman_table = {
#     0: "0001", 1: "0010", 2: "0011", 3: "0100", 4: "0101",
#     5: "0110", 6: "0111", 7: "1000"
# }

# # Example encoded data (run-length, code-length, Huffman code)
# encoded_data = [(1, 4, "0001"), (2, 3, "001"), (3, 5, "01000")]
encoding, hufftable = encode("coil-20-unproc/obj1__0.png", quant_matrix)
print(os.path.getsize("coil-20-unproc/obj1__0.png"))
# Writing to a file
write_compressed_file("compressed_image.dat", quant_matrix, hufftable, encoding)
filename = "compressed_image.dat"
file_size = os.path.getsize(filename)
print(f"Size of the file '{filename}': {file_size} bytes")

# Open the file in binary read mode
with open(filename, 'rb') as f:
    # Read the entire file content as bytes
    file_data = f.read()

# Print each byte in the file
# for byte in file_data:
#     print(f"{byte:08b}")  # Print the byte in binary format


[[128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]
 ...
 [128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]
 [128 128 128 ... 128 128 128]]
122.0
<class 'numpy.int64'>
<class 'numpy.int64'>
23384
Size of the file 'compressed_image.dat': 11844 bytes


- Loading from the file

In [ ]:
def read_compressed_file(filename):
    with open(filename, 'rb') as f:
        quant_matrix = np.load(f)
        huffman_table = np.load(f)
        encoded_data = np.load(f)
    return quant_matrix, huffman_table, encoded_data

- decoding run length encoding

In [ ]:
def decode_rle_data(encoded_bits, huffman_decoder):
    """
    Decode RLE encoded data to reconstruct the original quantized 8x8 DCT block.

    Parameters:
    - encoded_bits (str): Binary string containing RLE encoded data.
    - huffman_decoder (dict): Dictionary mapping Huffman codes to original coefficients.

    Returns:
    - np.array: Reconstructed 8x8 DCT block.
    """
    decoded_coefficients = []
    i = 0

    while i < len(encoded_bits):
        # Read 4 bits for run-length and 4 bits for size
        run_length = int(encoded_bits[i:i+4], 2)
        size = int(encoded_bits[i+4:i+8], 2)
        i += 8

        if size == 0:  # End-of-Block (EOB) marker
            break

        # Decode the Huffman code to get the coefficient
        huffman_code = ""
        while huffman_code not in huffman_decoder:
            huffman_code += encoded_bits[i]
            i += 1
        coefficient = huffman_decoder[huffman_code]

        # Add zeros for the run-length
        decoded_coefficients.extend([0] * run_length)
        decoded_coefficients.append(coefficient)

    # Fill remaining zeros to complete the block (8x8 = 64 elements)
    while len(decoded_coefficients) < 64:
        decoded_coefficients.append(0)

    # Reshape the list into an 8x8 block
    return np.array(decoded_coefficients).reshape(8, 8)

In [ ]:
image = np.random.randint(0, 10, (8, 8), dtype=np.uint8)

a, b = encode(image)
write_compressed_file("hello.ourjpeg", image, b, a)
qmat, hufftree, enc = read_compressed_file("hello.ourjpeg")

print(qmat)
print(hufftree)
print(enc)




TypeError: encode() missing 1 required positional argument: 'qmat'